# Find the Repos Available in your Database, and What Repository Groups They Are In

## Connect to your database

In [1]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
import sklearn 

warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [2]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived, 
                b.repo_git
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived,repo_git
0,3scale,25433,3scale,25639,Parent not available,0.0,https://github.com/3scale/3scale.github.io
1,3scale,25433,3scale-amp-openshift-templates,25613,Parent not available,0.0,https://github.com/3scale/3scale-amp-openshift...
2,3scale,25433,3scale-api-python,25662,3scale-qe/3scale-api-python,0.0,https://github.com/3scale/3scale-api-python
3,3scale,25433,3scale-api-ruby,25607,Parent not available,0.0,https://github.com/3scale/3scale-api-ruby
4,3scale,25433,3scale-go-client,25643,Parent not available,0.0,https://github.com/3scale/3scale-go-client
...,...,...,...,...,...,...,...
1325,quay,25430,test-cluster,25525,Parent not available,0.0,https://github.com/quay/test-cluster
1326,quay,25430,update-banner-job,25510,Parent not available,0.0,https://github.com/quay/update-banner-job
1327,quay,25430,update-py2-db,25517,Parent not available,0.0,https://github.com/quay/update-py2-db
1328,quay,25430,update-ro-keys-job,25518,Parent not available,0.0,https://github.com/quay/update-ro-keys-job


rg_name           object
repo_group_id      int64
repo_name         object
repo_id            int64
forked_from       object
repo_archived    float64
repo_git          object
dtype: object

## Adapted version of clustering-worker-tasks.py 

In [3]:
import logging
import os
import time
import traceback
import re
import pickle

import sqlalchemy as s
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation  as LDA
from collections import OrderedDict
from textblob import TextBlob
from collections import Counter
from sklearn.decomposition import TruncatedSVD as TSVD


# def clustering_model(repo_git: str,logger,engine, session) -> None:
def clustering_model(repo_id: int, engine) -> None:

    ngram_range = (1, 4)
    clustering_by_content = True
    clustering_by_mechanism = False

    # define topic modeling specific parameters
    num_topics = 20
    num_words_per_topic = 20

    tool_source = 'Clustering Worker'
    tool_version = '0.2.0'
    data_source = 'Augur Collected Messages'

    #query = session.query(Repo).filter(Repo.repo_git == repo_git)
    #repo_id = execute_session_query(query, 'one').repo_id

    num_clusters = 20
    max_df = 0.9
    max_features = 10000
    min_df = 0.1

    get_messages_for_repo_sql = s.sql.text(
        """
            SELECT
                r.repo_group_id,
                r.repo_id,
                r.repo_git,
                r.repo_name,
                i.issue_id thread_id,
                M.msg_text,
                i.issue_title thread_title,
                M.msg_id 
            FROM
                augur_data.repo r,
                augur_data.issues i,
                augur_data.message M,
                augur_data.issue_message_ref imr 
            WHERE
                r.repo_id = i.repo_id 
                AND imr.issue_id = i.issue_id 
                AND imr.msg_id = M.msg_id 
                AND r.repo_id=:repo_id
            UNION
            SELECT
                r.repo_group_id,
                r.repo_id,
                        r.repo_git,
                r.repo_name,
                pr.pull_request_id thread_id,
                M.msg_text,
                pr.pr_src_title thread_title,
                M.msg_id 
            FROM
                augur_data.repo r,
                augur_data.pull_requests pr,
                augur_data.message M,
                augur_data.pull_request_message_ref prmr 
            WHERE
                r.repo_id = pr.repo_id 
                AND prmr.pull_request_id = pr.pull_request_id 
                AND prmr.msg_id = M.msg_id 
                AND r.repo_id=:repo_id
            """
    )
    # result = db.execute(delete_points_SQL, repo_id=repo_id, min_date=min_date)
    msg_df_cur_repo = pd.read_sql(get_messages_for_repo_sql, engine, params={"repo_id": repo_id}) #changed repo id to augur's repo id in astros.
  
    # check if dumped pickle file exists, if exists no need to train the model
    if not os.path.exists(MODEL_FILE_NAME):
        #train_model(engine, session, max_df, min_df, max_features, ngram_range, num_clusters, num_topics, num_words_per_topic, tool_source, tool_version, data_source)
        train_model(engine, max_df, min_df, max_features, ngram_range, num_clusters, num_topics, num_words_per_topic, tool_source, tool_version, data_source)
    else:
        model_stats = os.stat(MODEL_FILE_NAME)
        model_age = (time.time() - model_stats.st_mtime)
        # if the model is more than month old, retrain it.
        if model_age > 2000000:
            # train_model(logger, engine, session, max_df, min_df, max_features, ngram_range, num_clusters, num_topics, num_words_per_topic, tool_source, tool_version, data_source)
            train_model(engine, max_df, min_df, max_features, ngram_range, num_clusters, num_topics, num_words_per_topic, tool_source, tool_version, data_source)
        else:
            pass

    with open("kmeans_repo_messages", 'rb') as model_file:
        kmeans_model = pickle.load(model_file)

    msg_df = msg_df_cur_repo.groupby('repo_id')['msg_text'].apply(','.join).reset_index()

    # logger.debug(f'messages being clustered: {msg_df}')

    if msg_df.empty:
        # logger.info("not enough data for prediction")
        # self.register_task_completion(task, repo_id, 'clustering')
        display('empty data frame.')
        return

    vocabulary = pickle.load(open("vocabulary", "rb"))

    tfidf_vectorizer = TfidfVectorizer(max_df=max_df, max_features=max_features,
                                       min_df=min_df, stop_words='english',
                                       use_idf=True, tokenizer=preprocess_and_tokenize,
                                       ngram_range=ngram_range, vocabulary=vocabulary)
    tfidf_transformer = tfidf_vectorizer.fit(
        msg_df['msg_text'])  # might be fitting twice, might have been used in training

    # save new vocabulary ??
    feature_matrix_cur_repo = tfidf_transformer.transform(msg_df['msg_text'])

    prediction = kmeans_model.predict(feature_matrix_cur_repo)

    # inserting data
    record = {
        #'repo_id': int(25613),
        'repo_id': int(repo_id),
        'cluster_content': int(prediction[0]),
        'cluster_mechanism': -1,
        'tool_source': tool_source,
        'tool_version': tool_version,
        'data_source': data_source
    }
    display(record)
    #repo_cluster_messages_obj = RepoClusterMessage(**record)
    #session.add(repo_cluster_messages_obj)
    #session.commit()

    # result = db.execute(repo_cluster_messages_table.insert().values(record))
    try:
        lda_model = pickle.load(open("lda_model", "rb"))
        vocabulary = pickle.load(open("vocabulary_count", "rb"))
        count_vectorizer = CountVectorizer(max_df=max_df, max_features=max_features, min_df=min_df,
                                           stop_words="english", tokenizer=preprocess_and_tokenize,
                                           vocabulary=vocabulary)
        count_transformer = count_vectorizer.fit(
            msg_df['msg_text'])  # might be fitting twice, might have been used in training

        # save new vocabulary ??
        count_matrix_cur_repo = count_transformer.transform(msg_df['msg_text'])
        prediction = lda_model.transform(count_matrix_cur_repo)

        for i, prob_vector in enumerate(prediction):
            # repo_id = msg_df.loc[i]['repo_id']
            for i, prob in enumerate(prob_vector):
                record = {
                    'repo_id': int(repo_id),
                    'topic_id': i + 1,
                    'topic_prob': prob,
                    'tool_source': tool_source,
                    'tool_version': tool_version,
                    'data_source': data_source
                }
                display(record)

                # repo_topic_object = RepoTopic(**record)
                #session.add(repo_topic_object)
                #session.commit()

                    # result = db.execute(repo_topic_table.insert().values(record))
    except Exception as e:
        stacker = traceback.format_exc()
        pass

    # self.register_task_completion(task, repo_id, 'clustering')


def get_tf_idf_matrix(text_list, max_df, max_features, min_df, ngram_range):

    tfidf_vectorizer = TfidfVectorizer(max_df=max_df, max_features=max_features,
                                       min_df=min_df, stop_words='english',
                                       use_idf=True, tokenizer=preprocess_and_tokenize,
                                       ngram_range=ngram_range)
    tfidf_transformer = tfidf_vectorizer.fit(text_list)
    tfidf_matrix = tfidf_transformer.transform(text_list)
    pickle.dump(tfidf_transformer.vocabulary_, open("vocabulary", 'wb'))
    return tfidf_matrix, tfidf_vectorizer.get_feature_names_out()

def cluster_and_label(feature_matrix, num_clusters):
    kmeans_model = KMeans(n_clusters=num_clusters)
    kmeans_model.fit(feature_matrix)
    pickle.dump(kmeans_model, open("kmeans_repo_messages", 'wb'))
    return kmeans_model.labels_.tolist()

def count_func(msg):
    blobed = TextBlob(msg)
    counts = Counter(tag for word, tag in blobed.tags if
                     tag not in ['NNPS', 'RBS', 'SYM', 'WP$', 'LS', 'POS', 'RP', 'RBR', 'JJS', 'UH', 'FW', 'PDT'])
    total = sum(counts.values())
    normalized_count = {key: value / total for key, value in counts.items()}
    return normalized_count

def preprocess_and_tokenize(text):
    text = text.lower()
    text = re.sub(r'[@]\w+', '', text)
    text = re.sub(r'[^A-Za-z]+', ' ', text)

    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if len(token) > 1]
    stems = [stemmer.stem(t) for t in tokens]
    return stems

def train_model(engine, max_df, min_df, max_features, ngram_range, num_clusters, num_topics, num_words_per_topic, tool_source, tool_version, data_source):
    # visualize_labels_PCA(tfidf_matrix.todense(), msg_df['cluster'], msg_df['repo_id'], 2, "tex!")

    #This error leads to an exploration of TruncatedSVD
    #Note that a precursor to the failure is there being a sparse matrix

    #File ~/github/virtualenvs/anotebook/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:485, in PCA._fit(self, X)
    #    479 if issparse(X):
    #    480     raise TypeError(
    #    481         "PCA does not support sparse input. See "
    #    482         "TruncatedSVD for a possible alternative."
    #    483     )
    #--> 485 X = self._validate_data(
    #    486     X, dtype=[np.float64, np.float32], ensure_2d=True, copy=self.copy
    #    487 )
    #    489 # Handle n_components==None
    #    490 if self.n_components is None:
        
    def visualize_labels_PCA(features, labels, annotations, num_components, title):
        labels_color_map = {-1: "red"}
        # added display call
        #display('number of components')
        #display(num_components)
        #display('features')
        #display(features)
        #display('labels')
        #display(labels)
        #display('annotations')
        #display(annotations)
        #display('titles')
        #display(title)
        for label in labels:
            labels_color_map[label] = [list([x / 255.0 for x in list(np.random.choice(range(256), size=3))])]
            
       # low_dim_data = PCA(n_components=num_components, svd_solver='full').fit_transform(features)
        low_dim_data = TSVD(n_components=num_components, n_iter=7, random_state=33).fit_transform(features)
        display(low_dim_data)

        fig, ax = plt.subplots(figsize=(20, 10))

        for i, data in enumerate(low_dim_data):
            pca_comp_1, pca_comp_2 = data
            color = labels_color_map[labels[i]]
            ax.scatter(pca_comp_1, pca_comp_2, c=color, label=labels[i])
        # ax.annotate(annotations[i],(pca_comp_1, pca_comp_2))

        handles, labels = ax.get_legend_handles_labels()
        handles_label_dict = OrderedDict(zip(labels, handles))
        ax.legend(handles_label_dict.values(), handles_label_dict.keys())

        plt.title(title)
        plt.xlabel("PCA Component 1")
        plt.ylabel("PCA Component 2")
        # plt.show()
        filename = labels + "_PCA.png"
        #display(filename)
        plt.save_fig(filename)

    get_messages_sql = s.sql.text(
        """
        SELECT r.repo_group_id, r.repo_id, r.repo_git, r.repo_name, i.issue_id thread_id,m.msg_text,i.issue_title thread_title,m.msg_id
        FROM augur_data.repo r, augur_data.issues i,
        augur_data.message m, augur_data.issue_message_ref imr
        WHERE r.repo_id=i.repo_id
        AND imr.issue_id=i.issue_id
        AND imr.msg_id=m.msg_id
        UNION
        SELECT r.repo_group_id, r.repo_id, r.repo_git, r.repo_name, pr.pull_request_id thread_id,m.msg_text,pr.pr_src_title thread_title,m.msg_id
        FROM augur_data.repo r, augur_data.pull_requests pr,
        augur_data.message m, augur_data.pull_request_message_ref prmr
        WHERE r.repo_id=pr.repo_id
        AND prmr.pull_request_id=pr.pull_request_id
        AND prmr.msg_id=m.msg_id
        """
    )
    msg_df_all = pd.read_sql(get_messages_sql, engine, params={})

    # select only highly active repos
    msg_df_all = msg_df_all.groupby("repo_id").filter(lambda x: len(x) > 100)

    # combining all the messages in a repository to form a single doc
    msg_df = msg_df_all.groupby('repo_id')['msg_text'].apply(','.join)
    msg_df = msg_df.reset_index()

    # dataframe summarizing total message count in a repositoryde
    message_desc_df = msg_df_all[["repo_id", "repo_git", "repo_name", "msg_id"]].groupby(
        ["repo_id", "repo_git", "repo_name"]).agg('count').reset_index()
    message_desc_df.columns = ["repo_id", "repo_git", "repo_name", "message_count"]

    tfidf_matrix, features = get_tf_idf_matrix(msg_df['msg_text'], max_df, max_features, min_df,
                                                    ngram_range)
    
    msg_df['cluster'] = cluster_and_label(tfidf_matrix, num_clusters)

    # LDA - Topic Modeling
    count_vectorizer = CountVectorizer(max_df=max_df, max_features=max_features, min_df=min_df,
                                       stop_words="english", tokenizer=preprocess_and_tokenize)

    # count_matrix = count_vectorizer.fit_transform(msg_df['msg_text'])
    count_transformer = count_vectorizer.fit(msg_df['msg_text'])
    count_matrix = count_transformer.transform(msg_df['msg_text'])
    pickle.dump(count_transformer.vocabulary_, open("vocabulary_count", 'wb'))
    feature_names = count_vectorizer.get_feature_names_out()

    # logger.debug("Calling LDA")
    lda_model = LDA(n_components=num_topics)
    lda_model.fit(count_matrix)
    # each component in lda_model.components_ represents probability distribution over words in that topic
    topic_list = lda_model.components_
    # Getting word probability
    # word_prob = lda_model.exp_dirichlet_component_
    # word probabilities
    # lda_model does not have state variable in this library
    # topics_terms = lda_model.state.get_lambda()
    # topics_terms_proba = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)
    # word_prob = [lda_model.id2word[i] for i in range(topics_terms_proba.shape[1])]

    # Site explaining main library used for parsing topics: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

    # Good site for optimizing: https://medium.com/@yanlinc/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6
    # Another Good Site: https://towardsdatascience.com/an-introduction-to-clustering-algorithms-in-python-123438574097
    # https://machinelearningmastery.com/clustering-algorithms-with-python/

    pickle.dump(lda_model, open("lda_model", 'wb'))

    ## Advance Sequence SQL

    # key_sequence_words_sql = s.sql.text(
    #                           """
    #       SELECT nextval('augur_data.topic_words_topic_words_id_seq'::text)
    #       """
    #                               )

    # twid = self.db.execute(key_sequence_words_sql)
    # logger.info("twid variable is: {}".format(twid))
    # insert topic list into database
    topic_id = 1
    for topic in topic_list:
        # twid = self.get_max_id('topic_words', 'topic_words_id') + 1
        # logger.info("twid variable is: {}".format(twid))
        for i in topic.argsort()[:-num_words_per_topic - 1:-1]:
            # twid+=1
            # logger.info("in loop incremented twid variable is: {}".format(twid))
            # logger.info("twid variable is: {}".format(twid))
            record = {
                # 'topic_words_id': twid,
                # 'word_prob': word_prob[i],
                'topic_id': int(topic_id),
                'word': feature_names[i],
                'tool_source': tool_source,
                'tool_version': tool_version,
                'data_source': data_source
            }
            
            
            ##############################
            #uncommenting the display things below will generate the topic words. Its long
            ##############################
            
            #display('topics')
            #display(record)

            # topic_word_obj = TopicWord(**record)
            #session.add(topic_word_obj)
            #session.commit()

            # result = db.execute(topic_words_table.insert().values(record))
            
        topic_id += 1

    # insert topic list into database

    # save the model and predict on each repo separately

    prediction = lda_model.transform(count_matrix)
    display(prediction)

    topic_model_dict_list = []
    for i, prob_vector in enumerate(prediction):
        topic_model_dict = {}
        topic_model_dict['repo_id'] = msg_df.loc[i]['repo_id']
        for i, prob in enumerate(prob_vector):
            topic_model_dict["topic" + str(i + 1)] = prob
        topic_model_dict_list.append(topic_model_dict)
    topic_model_df = pd.DataFrame(topic_model_dict_list)
    #display('topic model dataframe')
    #display(topic_model_df)

    result_content_df = topic_model_df.set_index('repo_id').join(message_desc_df.set_index('repo_id')) #.join(
        # msg_df.set_index('repo_id'))
    result_content_df = result_content_df.reset_index()
    
    
    result_content_df.to_csv('out.csv')
    
    try:
        POS_count_dict = msg_df.apply(lambda row: count_func(row['msg_text']), axis=1)
    except Exception as e:
        stacker = traceback.format_exc()
        pass
    try:
        msg_df_aug = pd.concat([msg_df, pd.DataFrame.from_records(POS_count_dict)], axis=1)
    except Exception as e:
        stacker = traceback.format_exc()
        pass
    
    try: 
        #display('entering visualize_labels_pca')
        
        np.asarray(tfidf_matrix)
        
        visualize_labels_PCA(tfidf_matrix.todense(), msg_df['cluster'], msg_df['repo_id'], 2, "tex!")
        #display('exiting visualize_labels_pca')
    except Exception as e:
        stacker = traceback.format_exc()
        display(stacker)
        pass
    
    #visualize_labels_PCA(tfidf_matrix.todense(), msg_df['cluster'], msg_df['repo_id'], 2, "MIN_DF={} and MAX_DF={} and NGRAM_RANGE={}".format(MIN_DF, MAX_DF, NGRAM_RANGE))


MODEL_FILE_NAME = "kmeans_repo_messages"
stemmer = nltk.stem.snowball.SnowballStemmer("english")

#clustering_model("https://github.com/chaoss/augur", engine, session)
#clustering_model("https://github.com/chaoss/augur", engine)
display('Done')

'Done'

# Next Steps: 
1. We now have output for the cluster number for each repo in a repository
2. How can we store the visualizations of these different repositories? 
3. Then, once stored, how can we visualize the repositories in their clusters?
4. How, separate from the clusters, does the weighting of each topic per repo look?
5. Perhaps, even, some tables to see the combination of clusters, and topic probabilities 
6. **Very useful**: This would be some research on the libraries, but creating documentation of the different methods, and parameters we path indicating what they do would be helpful for consumers of the data. 

In [4]:
# repolist comes from the second cell executed in this notebook
for index, row in repolist.iterrows():
    print(row.repo_id)
    thisrepoid = row.repo_id
    this_git_url = row.repo_git
    print(row.repo_git)
    clustering_model(thisrepoid, engine)

25639
https://github.com/3scale/3scale.github.io


array([[6.99594252e-07, 6.99594252e-07, 6.99594252e-07, ...,
        6.99594250e-07, 6.99594255e-07, 6.99594252e-07],
       [5.44662322e-05, 5.44662319e-05, 5.44662321e-05, ...,
        5.44662319e-05, 5.44662321e-05, 7.96298129e-02],
       [6.91662987e-02, 1.50150153e-04, 4.35500783e-02, ...,
        1.50150153e-04, 1.50150153e-04, 1.50150154e-04],
       ...,
       [5.29997893e-06, 9.81764692e-03, 4.95122469e-02, ...,
        5.29997890e-06, 5.29997891e-06, 2.19565981e-03],
       [6.03646031e-06, 6.03646031e-06, 6.03646029e-06, ...,
        6.03646032e-06, 6.03646028e-06, 6.03646032e-06],
       [1.26008065e-05, 1.26008066e-05, 1.26008065e-05, ...,
        1.26008065e-05, 1.26008065e-05, 1.26008065e-05]])

'Traceback (most recent call last):\n  File "C:\\Users\\NYAN_\\AppData\\Local\\Temp\\ipykernel_29672\\701579419.py", line 434, in train_model\n    visualize_labels_PCA(tfidf_matrix.todense(), msg_df[\'cluster\'], msg_df[\'repo_id\'], 2, "tex!")\n  File "C:\\Users\\NYAN_\\AppData\\Local\\Temp\\ipykernel_29672\\701579419.py", line 258, in visualize_labels_PCA\n    low_dim_data = TSVD(n_components=num_components, n_iter=7, random_state=33).fit_transform(features)\n  File "C:\\Users\\NYAN_\\virtualenvs\\augur-community-reports\\lib\\site-packages\\sklearn\\utils\\_set_output.py", line 140, in wrapped\n    data_to_wrap = f(self, X, *args, **kwargs)\n  File "C:\\Users\\NYAN_\\virtualenvs\\augur-community-reports\\lib\\site-packages\\sklearn\\decomposition\\_truncated_svd.py", line 224, in fit_transform\n    X = self._validate_data(X, accept_sparse=["csr", "csc"], ensure_min_features=2)\n  File "C:\\Users\\NYAN_\\virtualenvs\\augur-community-reports\\lib\\site-packages\\sklearn\\base.py", lin

{'repo_id': 25639,
 'cluster_content': 7,
 'cluster_mechanism': -1,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}

{'repo_id': 25639,
 'topic_id': 1,
 'topic_prob': 0.418255677848206,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}

...

{'repo_id': 25507,
 'topic_id': 16,
 'topic_prob': 0.004545454674836219,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}

{'repo_id': 25507,
 'topic_id': 17,
 'topic_prob': 0.0045454546388552,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}

{'repo_id': 25507,
 'topic_id': 18,
 'topic_prob': 0.0045454545700501555,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}

{'repo_id': 25507,
 'topic_id': 19,
 'topic_prob': 0.004545454594468969,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}

{'repo_id': 25507,
 'topic_id': 20,
 'topic_prob': 0.004545454611431568,
 'tool_source': 'Clustering Worker',
 'tool_version': '0.2.0',
 'data_source': 'Augur Collected Messages'}